# 그래프 그리기

- Plotly (https://plotly.com/python/)


## 예제


In [1]:
import plotly.express as px

# 예제 데이터셋 로드
df = px.data.stocks()
df

,date,GOOG,AAPL,AMZN,FB,NFLX,MSFT
0,2018-01-01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,2018-01-08,1.018172,1.011943,1.061881,0.959968,1.053526,1.015988
2,2018-01-15,1.032008,1.019771,1.053240,0.970243,1.049860,1.020524
3,2018-01-22,1.066783,0.980057,1.140676,1.016858,1.307681,1.066561
4,2018-01-29,1.008773,0.917143,1.163374,1.018357,1.273537,1.040708
...,...,...,...,...,...,...,...
100,2019-12-02,1.216280,1.546914,1.425061,1.075997,1.463641,1.720717
101,2019-12-09,1.222821,1.572286,1.432660,1.038855,1.421496,1.752239
102,2019-12-16,1.224418,1.596800,1.453455,1.104094,1.604362,1.784896
103,2019-12-23,1.226504,1.656000,1.521226,1.113728,1.567170,1.802472


In [2]:
fig = px.line(df, x="date", y="GOOG")
fig.show()

In [3]:
import plotly.express as px

df = px.data.stocks(indexed=True)
df

company,GOOG,AAPL,AMZN,FB,NFLX,MSFT
date,,,,,,
2018-01-01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2018-01-08,1.018172,1.011943,1.061881,0.959968,1.053526,1.015988
2018-01-15,1.032008,1.019771,1.053240,0.970243,1.049860,1.020524
2018-01-22,1.066783,0.980057,1.140676,1.016858,1.307681,1.066561
2018-01-29,1.008773,0.917143,1.163374,1.018357,1.273537,1.040708
...,...,...,...,...,...,...
2019-12-02,1.216280,1.546914,1.425061,1.075997,1.463641,1.720717
2019-12-09,1.222821,1.572286,1.432660,1.038855,1.421496,1.752239
2019-12-16,1.224418,1.596800,1.453455,1.104094,1.604362,1.784896


In [4]:
fig = px.area(df, facet_col="company", facet_col_wrap=2)
fig.show()

# 2 캔들차트


In [1]:
def stockDataReader_fn(stock_name, start_date=None, end_date=None):
    import FinanceDataReader as fdr
    import datetime
    import os
    import pandas as pd

    # 1. 저장 파일 형식에 필요한 Data 불러오기
    today_year = datetime.datetime.today().year
    today_month = datetime.datetime.today().month
    today_day = datetime.datetime.today().day

    # 2. KRX 종목 DataFrame 존재 여부 확인
    csv_path = "krx_df_{}_{}_{}.csv".format(today_year, today_month, today_day)

    if os.path.isfile(csv_path):
        krx_df = pd.read_csv(csv_path)
    else:
        krx_df = fdr.StockListing("KRX")
        krx_df.to_csv(csv_path)

    # 3. 종목 코드 찾기
    if stock_name in krx_df["Name"].values:
        stock_symbol = krx_df[krx_df["Name"] == stock_name]["Code"].values[0]
    else:
        print("해당 종목이 없습니다.")
        raise "해당 종목이 없습니다."

    stock_df = fdr.DataReader(stock_symbol, start=start_date, end=end_date)
    return stock_df

In [17]:
stock_name = "삼성전자"
start_date = "2025"

df = stockDataReader_fn(stock_name, start_date=start_date)
df

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2025-01-02,52700,53600,52300,53400,16630538,0.003759
2025-01-03,52800,55100,52800,54400,19318046,0.018727
2025-01-06,54400,56200,54300,55900,19034284,0.027574
2025-01-07,56800,57300,55400,55400,17030235,-0.008945
2025-01-08,54800,57500,54700,57300,26593553,0.034296
...,...,...,...,...,...,...
2026-02-06,154100,160300,151600,158600,36358081,-0.004394
2026-02-09,168600,168700,165000,166400,25098971,0.049180
2026-02-10,167400,168100,165500,165800,19369566,-0.003606


In [18]:
import plotly.graph_objects as go

candlestick = go.Candlestick(
    x=df.index,
    open=df["Open"],
    high=df["High"],
    low=df["Low"],
    close=df["Close"],
    name="CandleStick",
)
fig = go.Figure(data=[candlestick])
fig.update_xaxes(rangeslider_visible=False)
fig.show()

# DrawDown 그리기


In [19]:
def mdd_fn(df):
    df = df[["Close"]].copy()

    def return_fn(df):
        return df["Close"].pct_change().fillna(0)

    def cum_return_fn(df_return):
        return (1 + df_return).cumprod()

    df["Return"] = return_fn(df)
    df["CumReturn"] = cum_return_fn(df["Return"])
    df["MaxCumReturn"] = df["CumReturn"].cummax()
    df["DrawDown"] = (df["CumReturn"] / df["MaxCumReturn"]) - 1
    mdd = df["DrawDown"].min()
    list_info = []
    min_close_value = df["Close"].min()
    max_close_value = df["Close"].max()

    print(
        "최고: {}\n최저: {}\nMDD: {}".format(
            max_close_value, min_close_value, round(mdd * 100, 2)
        )
    )
    df_max_close = df[df["DrawDown"] == 0].copy()
    df_max_close.loc[df.index[len(df) - 1]] = 0
    period = df_max_close.index[1:] - df_max_close.index[:-1]
    mdd_days = period.days
    max_period = mdd_days.max()
    max_period_idx = mdd_days.argmax()
    print(
        "MAX Draw Down: {} ~ {}".format(
            df_max_close.index[:-1][max_period_idx].date(),
            df_max_close.index[1:][max_period_idx].date(),
        )
    )
    print("{} days".format(max_period))
    print("max period : {}".format(max_period))

    list_info.append(max_close_value)
    list_info.append(min_close_value)
    list_info.append(round(mdd * 100, 2))
    list_info.append(df_max_close.index[:-1][max_period_idx].date())
    list_info.append(df_max_close.index[1:][max_period_idx].date())
    list_info.append(max_period)

    return df, list_info


In [20]:
df_dd, stock_info = mdd_fn(df)
df_dd

최고: 178600
최저: 51000
MDD: -14.67
MAX Draw Down: 2025-03-27 ~ 2025-07-03
98 days
max period : 98


,Close,Return,CumReturn,MaxCumReturn,DrawDown
Date,,,,,
2025-01-02,53400,0.000000,1.000000,1.000000,0.000000
2025-01-03,54400,0.018727,1.018727,1.018727,0.000000
2025-01-06,55900,0.027574,1.046816,1.046816,0.000000
2025-01-07,55400,-0.008945,1.037453,1.046816,-0.008945
2025-01-08,57300,0.034296,1.073034,1.073034,0.000000
...,...,...,...,...,...
2026-02-06,158600,-0.004394,2.970037,3.166667,-0.062093
2026-02-09,166400,0.049180,3.116105,3.166667,-0.015967
2026-02-10,165800,-0.003606,3.104869,3.166667,-0.019515


In [21]:
import plotly.graph_objects as go

dd = go.Scatter(
    x=df_dd.index,
    y=df_dd["DrawDown"],
    name="Draw Down",
    fill="tozeroy",
)
fig = go.Figure(data=[dd])
fig.show()

# MACD 그리기


In [14]:
def macdOscillator_fn(df, short_N, long_N, signal_N):
    df = df[["Close"]].copy()

    df["Short"] = df["Close"].ewm(span=short_N, adjust=False).mean()
    df["Long"] = df["Close"].ewm(span=long_N, adjust=False).mean()
    df["MACD"] = df["Short"] - df["Long"]
    df["Signal"] = df["MACD"].ewm(span=signal_N, adjust=False).mean()
    df["MACD Oscillator"] = df["MACD"] - df["Signal"]

    return df[["MACD", "Signal", "MACD Oscillator"]]

In [22]:
df_macd = macdOscillator_fn(df, short_N=9, long_N=26, signal_N=13)
df_macd

,MACD,Signal,MACD Oscillator
Date,,,
2025-01-02,0.000000,0.000000,0.000000
2025-01-03,125.925926,17.989418,107.936508
2025-01-06,406.227709,73.452031,332.775678
2025-01-07,544.877509,140.798528,404.078981
2025-01-08,878.768063,246.222747,632.545316
...,...,...,...
2026-02-06,11115.712105,12282.166734,-1166.454629
2026-02-09,11190.373366,12126.196253,-935.822887
2026-02-10,11004.339138,11965.930951,-961.591812


In [23]:
import plotly.graph_objects as go

macd = go.Scatter(x=df.index, y=df_macd["MACD"], name="MACD")
signal = go.Scatter(x=df.index, y=df_macd["Signal"], name="Signal")
oscillator = go.Bar(x=df.index, y=df_macd["MACD Oscillator"], name="MACD Oscillator")

fig = go.Figure(data=[macd, signal, oscillator])
fig.show()